In [1]:
import sqlite3
import pandas as pd

In [2]:
connect = sqlite3.connect('responsi.db')
cursor = connect.cursor()

In [3]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Mahasiswa(
        NIM int(11) PRIMARY KEY NOT NULL,
        Nama TEXT,
        TanggalLahir DATE,
        Alamat TEXT,
        Password TEXT
    )
''')

In [4]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Dosen(
        NIDN int(11) PRIMARY KEY NOT NULL,
        Nama TEXT,
        Departemen TEXT,
        Email TEXT
    )
''')

In [5]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS MataKuliah(
        KodeMK VARCHAR(11) PRIMARY KEY,
        NamaMK TEXT,
        SKS TEXT,
        NIDN int(11) NOT NULL,
        FOREIGN KEY (NIDN) REFERENCES Dosen(NIDN)
    )
''')

In [6]:
query = 'SELECT * FROM Mahasiswa'

data = pd.read_sql_query(query, connect)
data

,NIM,Nama,TanggalLahir,Alamat,Password


In [7]:
import struct

k = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

# Define and assign initial hash values inside the function
h0=0x6a09e667
h1=0xbb67ae85
h2=0x3c6ef372
h3=0xa54ff53a
h4=0x510e527f
h5=0x9b05688c
h6=0x1f83d9ab
h7=0x5be0cd19

a = h0
b = h1
c = h2
d = h3
e = h4
f = h5
g = h6
h = h7

def intToendian(num):
    return num.to_bytes(4, byteorder='little')

def endian_length(length):
    return length.to_bytes(8, byteorder='little')

def left_rotate(x, y):
    return ((x << y) | (x >> (32 - y))) & 0xFFFFFFFF


def generate_SHA256(message):
    real_width = len(message)
    message += b'\x80'
    while len(message) % 64 != 56:
        message += b'\x80'
    message += endian_length(real_width * 8)
    a=h0
    b=h1
    c=h2
    d=h3
    e=h4
    f=h5
    g=h6
    h=h7

    for i in range(0, len(message), 64):
        chunk = message[i:i+64]
        w = [0] * 64
        for j in range(16):
            w[j] = struct.unpack('<I', chunk[j*4:j*4+4])[0]
        for j in range(16, 64):
            s0      = left_rotate(w[j-15], 7) ^ left_rotate(w[j-15], 18) ^ (w[j-15] >> 18)
            s1      = left_rotate(w[j-2], 17) ^ left_rotate(w[j-2], 19) ^ (w[j-2] >> 10)
            w[j]    = (w[j-16] + s0 + w[j-7] + s1) & 0xFFFFFFFF
        a_temp = a
        b_temp = b
        c_temp = c
        d_temp = d
        e_temp = e
        f_temp = f
        g_temp = g
        h_temp = h

        for j in range(64):
            S1      = left_rotate(e_temp, 6) ^ left_rotate(e_temp, 11) ^ left_rotate(e_temp, 25)
            ch      = (e_temp + f_temp) ^ (-e_temp & g_temp)
            temp1   = (h_temp + S1 + ch + k[j] + w[j]) & 0xFFFFFFFF

            S0      = left_rotate(a_temp, 2) ^ left_rotate(a_temp, 13) ^ left_rotate(a_temp, 22)
            maj     = (a_temp & b_temp) ^ (a_temp & c_temp) ^ (b_temp & c_temp)
            temp2   = (S0 + maj) & 0xFFFFFFFF

            h_temp = g_temp
            g_temp = f_temp
            f_temp = e_temp
            e_temp = (d_temp + temp1) & 0xFFFFFFFF
            d_temp = c_temp
            c_temp = b_temp
            b_temp = a_temp

            a_temp = (temp1 + temp2) & 0xFFFFFFFF
        
        a = (a + a_temp) & 0xFFFFFFFF
        b = (b + b_temp) & 0xFFFFFFFF
        c = (c + c_temp) & 0xFFFFFFFF
        d = (d + d_temp) & 0xFFFFFFFF
        e = (e + e_temp) & 0xFFFFFFFF
        f = (f + f_temp) & 0xFFFFFFFF
        g = (g + g_temp) & 0xFFFFFFFF
        h = (h + h_temp) & 0xFFFFFFFF

    hash_result = (a << 224) | (b << 192) | (c << 160) | (d << 128) | (e << 96) | (f << 64) | (g << 32) | h

    return '{:08x}'.format(hash_result)

**HASHING PASSWORD SHA256**

Alasan meng-generate password ke SHA256 yaitu password tidak disimpan dalam bentuk yang mudah dibaca 
dan dipahami, sehingga meningkatkan keamanan data mahasiswa dalam database.

In [8]:
Mahasiswa = [
    (5220411100, 'John Doe', '2000-05-15', '123 Main St', 'pass123'),
    (5220411101, 'Jane Smith', '1999-10-20', '456 Elm St', 'abcdef'),
    (5220411102, 'Michael Johnson', '2001-03-25', '789 Oak St', '123456'),
    (5220411103, 'Sarah Lee', '2002-08-12', '101 Pine St', 'qwerty'),
    (5220411104, 'David Brown', '2000-12-30', '202 Maple St', 'pass123')
]

def generate(password):
    result = generate_SHA256(password)
    return result

for data in Mahasiswa:
    password_plain = data[4]
    hashed_password = generate(password_plain.encode('utf-8'))
    hashed_data = data[:4] + (hashed_password,)
    print(f"SHA256: {password_plain}  >  {hashed_password}")
    
    cursor.execute('''
    INSERT INTO Mahasiswa(
        NIM,
        Nama,
        TanggalLahir,
        Alamat,
        Password) VALUES (?,?,?,?,?)''', hashed_data)

connect.commit()

SHA256: pass123  >  48a6d1c7081661aacc2794b412ae36e596c46d693e3f2f6671f27262b76ca800
SHA256: abcdef  >  b633c0326ee0a50a997544c77e5c76ff52eece99418745f7427d7e7baf03f87b
SHA256: 123456  >  5b26e94f83805b618c1ea796af86dd5134fbf4a842a6592cf73345b2d4796cb2
SHA256: qwerty  >  bac524e7658cbf186db504743a87a7a0631409c617de432a69b487b377ad0958
SHA256: pass123  >  48a6d1c7081661aacc2794b412ae36e596c46d693e3f2f6671f27262b76ca800


In [9]:
Dosen = [
    (121547001, 'Andi Susanto', 'Teknik Informatika', 'andi.susanto@example.com'),
    (121547002, 'Dewi Indah', 'Ekonomi Pembangunan', 'dewi.indah@example.com'),
    (121547003, 'Rudi Pratama', 'Hukum Tata Negara', 'rudi.pratama@example.com')

]

for data in Dosen:
    cursor.execute('''
    INSERT INTO Dosen(
        NIDN,
        Nama,
        Departemen,
        Email) VALUES (?,?,?,?)''', data)

connect.commit()

In [10]:
MataKuliah = [
    ('IF101', 'Pemrograman Dasar', '3', 121547001),
    ('EK201', 'Ekonomi Mikro', '3', 121547002),
    ('HI301', 'Sejarah Indonesia', '3', 121547003),
    ('MG401', 'Manajemen Sumber Daya Manusia', '3', 121547001)
]

for data in MataKuliah:
    cursor.execute('''
    INSERT INTO MataKuliah(
        KodeMK,
        NamaMK,
        SKS,
        NIDN) VALUES (?,?,?,?)''', data)

connect.commit()

In [11]:
query = 'SELECT * FROM Mahasiswa'

data = pd.read_sql_query(query, connect)
data

,NIM,Nama,TanggalLahir,Alamat,Password
0,5220411100,John Doe,2000-05-15,123 Main St,48a6d1c7081661aacc2794b412ae36e596c46d693e3f2f...
1,5220411101,Jane Smith,1999-10-20,456 Elm St,b633c0326ee0a50a997544c77e5c76ff52eece99418745...
2,5220411102,Michael Johnson,2001-03-25,789 Oak St,5b26e94f83805b618c1ea796af86dd5134fbf4a842a659...
3,5220411103,Sarah Lee,2002-08-12,101 Pine St,bac524e7658cbf186db504743a87a7a0631409c617de43...
4,5220411104,David Brown,2000-12-30,202 Maple St,48a6d1c7081661aacc2794b412ae36e596c46d693e3f2f...


In [12]:
query = 'SELECT * FROM Dosen'

data = pd.read_sql_query(query, connect)
data

,NIDN,Nama,Departemen,Email
0,121547001,Andi Susanto,Teknik Informatika,andi.susanto@example.com
1,121547002,Dewi Indah,Ekonomi Pembangunan,dewi.indah@example.com
2,121547003,Rudi Pratama,Hukum Tata Negara,rudi.pratama@example.com


In [13]:
query = 'SELECT * FROM MataKuliah'

data = pd.read_sql_query(query, connect)
data

,KodeMK,NamaMK,SKS,NIDN
0,IF101,Pemrograman Dasar,3,121547001
1,EK201,Ekonomi Mikro,3,121547002
2,HI301,Sejarah Indonesia,3,121547003
3,MG401,Manajemen Sumber Daya Manusia,3,121547001
